In [221]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt, mpld3
from sklearn.linear_model import LassoCV
from sklearn.linear_model import LogisticRegressionCV
from sklearn.feature_selection import SelectFromModel
import plotly.graph_objects as go
import os

In [161]:
np.random.seed(12345)

In [162]:
# Download the latest SO dataset here: https://drive.google.com/open?id=1QOmVDpd8hcVYqqUXDXf68UMDWQZP0wQV
data_2019 = pd.read_csv("developer_survey_2019/survey_results_public.csv", low_memory=False)

In [163]:
to_drop = ["Respondent", "OpenSource", "CareerSat", "JobSat", "JobSeek", "ResumeUpdate", "SurveyLength", "SurveyEase",
    "WelcomeChange", "EntTeams", "ScreenName", "LastIn", "SO", "Blockchain", "WorkChallenge", "BetterLife", "OffOn", "Currency",
    "CompTotal", "CompFreq", "MainBranch", "PlatformDesireNextYear", "LanguageDesireNextYear", "DatabaseDesireNextYear", 
    "MiscTechDesireNextYear", "WebFrameDesireNextYear", "MgrMoney", "ITperson", "Age1stCode", "MgrIdiot", "MgrWant"]

In [164]:
def col_drop(df: pd.DataFrame, to_drop: list) -> pd.DataFrame:

    df_dropped = df.copy()

    for flag in to_drop:
        try:
            df_dropped.drop([x for x in df_dropped.columns if flag in x], axis=1, inplace=True)
        except:
            pass

    return df_dropped

In [165]:
def string_replace(s: str) -> int:
    try:
        s = float(s)
    except:
        s = -1000

    return s

In [166]:
def text_clean(text: str) -> str:
    text = str(text).replace(" ", "_").replace("-", "_").replace(
        ",", "_").replace(".", "").replace("+", "p").replace("#", "s").replace(
            "/", "_").replace("'", "").replace("ʼ", "").replace(
                "(", "_").replace(")", "_").replace("’", "").replace(
                    "__", "_").replace("__", "_").replace("“", "").replace(
                        "”", "").replace(":", "_").replace("&", "_").lower()

    banned = ["participated_in_", "_or", "_of", "_etc", "_employees", "taken_", "_african_descent", "employed_", "developer_"]

    for t in banned: text = text.replace(t, "")

    return text

In [167]:
def create_controls(df: pd.DataFrame, exclude: str) -> dict:

    controls = {}

    for col in df.columns:
        if col != exclude:
            controls[col] = {
                "omitted": text_clean(pd.Series([x for sub in list(
                           data_2019[col]
                           .apply(text_clean)
                           .apply(lambda x: str(x).split(";"))) for x in sub])
                           .value_counts()
                           .idxmax()), 
                "controls": list(set([x for sub in list(
                            data_2019[col]
                            .apply(text_clean)
                            .apply(lambda x: str(x).split(";"))) for x in sub]))}

    return controls

In [168]:
def design_matrix(df: pd.DataFrame, controls: dict) -> pd.DataFrame:
    dm = df.copy()
    
    for control in controls.keys():
        dm[control] = dm[control].apply(text_clean)

        if control in num_columns:
            for c in controls[control]["controls"]:
                dm[control+"_"+c] = (dm[control] == c) * 1

        else:
            for c in controls[control]["controls"]:
                dm[control+"_"+c] = dm[control].apply(lambda x: c in str(x).split(";")) * 1

        dm.drop(control, axis=1, inplace=True)
        dm.drop(control+"_"+controls[control]["omitted"], axis=1, inplace=True)

    dm = dm[sorted(dm.columns)]

    #dm = dm[dm["DevType_student"] == 0]

    #dm = dm[x for x in dm.columns if "DevType_student" not in x]

    return dm

In [169]:
data_2019 = col_drop(data_2019, to_drop)

In [170]:
# Only consider those with income between $10,000 and $250,000
data_2019 = data_2019[(data_2019["ConvertedComp"] >= 10000) & (data_2019["ConvertedComp"] <= 250000)]
data_2019["ConvertedComp"] = np.log(data_2019["ConvertedComp"])
data_2019 = data_2019.rename(columns = {"ConvertedComp": "Income"})

# Only consider US respondents
data_2019 = data_2019[data_2019["Country"] == "United States"]
data_2019.drop(["Country"], axis=1, inplace=True)

# Only consider 18+ respondents
data_2019 = data_2019[data_2019["Age"] >= 21]

# Only consider respondents in the workforce
data_2019 = data_2019[data_2019["Employment"] != "Retired"]
data_2019 = data_2019[data_2019["Employment"] != "Not employed, and not looking for work"]

data_2019 = data_2019[data_2019["WorkWeekHrs"] >= 5]

# No students
data_2019 = data_2019[["Student" not in str(x) for x in data_2019["DevType"].values]]
data_2019 = data_2019[data_2019["Student"] == "No"]

# Only consider those with at least some education
data_2019 = data_2019[data_2019["EdLevel"] != "I never completed any formal education"]

data_2019 = data_2019.fillna("no_answer")

num_columns = ["Age", "YearsCode", "YearsCodePro", "WorkWeekHrs", "CodeRevHrs"]

# Convert numeric columns to int
for col in num_columns:
    data_2019[col] = data_2019[col].astype("int32", errors="ignore")

#data_2019["Age1stCode"].replace("Younger than 5 years", "4", inplace=True)
data_2019["YearsCode"].replace("Less than 1 year", "0", inplace=True)
data_2019["YearsCode"].replace("More than 50 years", "51", inplace=True)

for col in num_columns:
    data_2019[col] = data_2019[col].apply(string_replace)

# Exclude respondents who selected multiple gender, race, or sexual orientation
# options
data_2019 = data_2019[~data_2019["Gender"].str.contains(";")]
data_2019 = data_2019[~data_2019["Ethnicity"].str.contains(";")]
data_2019 = data_2019[~data_2019["Sexuality"].str.contains(";")]

# Reset index
data_2019 = data_2019.reset_index(drop=True)

In [171]:
num_labels = [
    ["no_answer", "21-25", "26-30", "31-35", "35-40", "41-45", "45-50", "51-55", "55-60", "61-65", "66-"],
    ["no_answer","00-05", "06-10", "11-15", "16-20", "21-25", "26-30", "31-35", "35-40", "41-"],
    ["no_answer","00-05", "06-10", "11-15", "16-20", "21-25", "26-30", "31-35", "35-40", "41-"],
    ["no_answer","-10", "11-20", "21-30", "31-40", "41-50", "51-60", "61-70", "71-80", "81-"],
    ["no_answer","01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11-15", "16-20", "21-"]
]

num_buckets = [
    np.array([-1001,0,25,30,35,40,45,50,55,60,65,100]),
    np.array([-1001,0,5,10,15,20,25,30,35,40,100]),
    np.array([-1001,0,5,10,15,20,25,30,35,40,100]),
    np.array([-1001,0,10,20,30,40,50,60,70,80,200]),
    np.array([-1001,0,1,2,3,4,5,6,7,8,9,10,15,20,200])
]

In [172]:
for i, col in enumerate(num_columns):
    data_2019[col] = pd.cut(data_2019[col], 
                            num_buckets[i], 
                            labels=num_labels[i]
                            ).astype("str")

In [173]:
controls = create_controls(data_2019, "Income")

In [174]:
data_2019 = design_matrix(data_2019, controls)

In [175]:
est = sm.OLS(endog=data_2019["Income"].values, 
             exog=data_2019.drop("Income", axis=1).assign(const=1)).fit()

In [183]:
est.summary()

-0.776      0.438      -0.038       0.016
PlatformWorkedWith_aws                                                                                            0.0230      0.008      2.931      0.003       0.008       0.038
PlatformWorkedWith_docker                                                                                         0.0066      0.009      0.707      0.479      -0.012       0.025
PlatformWorkedWith_google_cloud_platform                                                                          0.0180      0.011      1.694      0.090      -0.003       0.039
PlatformWorkedWith_heroku                                                                                         0.0042      0.011      0.374      0.708      -0.018       0.026
PlatformWorkedWith_ibm_cloud_watson                                                                              -0.0264      0.024     -1.083      0.279      -0.074       0.021
PlatformWorkedWith_ios                                                                                            0.0233      0.014      1.612      0.107      -0.005       0.052
PlatformWorkedWith_kubernetes                                                                                     0.0003      0.012      0.024      0.981      -0.023       0.023
PlatformWorkedWith_macos                                                                                          0.0148      0.008      1.757      0.079      -0.002       0.031
PlatformWorkedWith_microsoft_azure                                                                                0.0423      0.010      4.192      0.000       0.023       0.062
PlatformWorkedWith_no_answer                                                                                      0.0155      0.013      1.212      0.226      -0.010       0.041
PlatformWorkedWith_other_s__                                                                                      0.0083      0.016      0.518      0.605      -0.023       0.040
PlatformWorkedWith_raspberry_pi                                                                                  -0.0049      0.010     -0.487      0.626      -0.025       0.015
PlatformWorkedWith_slack                                                                                          0.0133      0.009      1.543      0.123      -0.004       0.030
PlatformWorkedWith_windows                                                                                       -0.0116      0.009     -1.356      0.175      -0.028       0.005
PlatformWorkedWith_wordpress                                                                                     -0.0342      0.011     -3.095      0.002      -0.056      -0.013
PurchaseHow_developers_typically_have_the_most_influence_on_purchasing_new_technology                             0.0245      0.009      2.663      0.008       0.006       0.043
PurchaseHow_no_answer                                                                                             0.0125      0.019      0.664      0.507      -0.024       0.049
PurchaseHow_not_sure                                                                                              0.0193      0.009      2.039      0.041       0.001       0.038
PurchaseHow_the_cto_cio_other_management_purchase_new_technology_typically_without_the_involvement_developers    -0.0083      0.010     -0.867      0.386      -0.027       0.010
PurchaseWhat_i_have_a_great_deal_influence                                                                        0.0198      0.010      2.000      0.046       0.000       0.039
PurchaseWhat_i_have_little_no_influence                                                                          -0.0329      0.008     -4.138      0.000      -0.048      -0.017
PurchaseWhat_no_answer                                                                                            0.0947      0.041      2.321      0.020       0.015       0.175
Sexuality_bisexual                        

In [252]:
def double_selection(df: pd.DataFrame, 
                     controls: dict, 
                     controls_adj: list, 
                     category: str) -> pd.DataFrame:

    X = df[controls_adj].copy()

    W = df.drop(controls_adj, axis=1).assign(const=1).copy()

    W = W.drop("Income", axis=1)

    Y = df["Income"].copy()

    clf = LassoCV(cv=5, max_iter=10000, selection="random", n_jobs=-1)

    sfm = SelectFromModel(clf)

    for i, X_j in enumerate(X.columns):
        if i==0:
            A = sfm.fit(W, X[X_j]).get_support()
        else:
            A = A | sfm.fit(W, X[X_j]).get_support()

    B = sfm.fit(W, Y).get_support()

    return pd.concat([W.T[A | B].T, X], axis=1)

In [217]:
def output_graph(df: pd.DataFrame, 
                 controls: dict, 
                 cov_selection: bool=False, 
                 category: str=None, 
                 category_plain: str=None, 
                 title: str=None) -> (pd.DataFrame, sm.regression.linear_model.RegressionResultsWrapper):

    controls_adj = [category+"_"+control for control in controls[category]["controls"] if controls[category]["omitted"] != control]

    if cov_selection:
        exog = double_selection(df, controls, controls_adj, category).assign(const=1)
    else:
        exog = df.drop("Income", axis=1).assign(const=1)

    est_full = sm.OLS(endog=df["Income"], 
                      exog=exog).fit()
    
    est_simple = sm.OLS(endog=df["Income"], 
                        exog=df.drop("Income", 
                        axis=1)[controls_adj]
                        .assign(const=1)).fit()
    
    output_full = pd.concat([est_full.params, est_full.bse*1.96], axis=1).rename(columns={0: "coef_full", 1: "conf_95_full"}).filter(like=category+"_", axis=0)

    output_simple = pd.concat([est_simple.params, est_simple.bse*1.96], axis=1).rename(columns={0: "coef_simple", 1: "conf_95_simple"}).filter(like=category+"_", axis=0)
    
    output = pd.concat([output_full, output_simple], axis=1)

    output.index = [x.replace(category + "_", "") for x in output.index]

    if category in num_columns:
        output.sort_index(axis=0, 
                          ascending=True, 
                          inplace=True)
    else:
        output.sort_values(by="coef_full", 
                           ascending=False, 
                           inplace=True)

    if category == "OrgSize":
        output = output.T[['just_me_i_am_a_freelancer_sole_proprietor', '2_9', '10_to_19', '20_to_99', '100_to_499', 
                         '500_to_999', '1_000_to_4_999', '5_000_to_9_999', 'no_answer']].T
    
    output["explained"] = output["coef_simple"] - output["coef_full"]

    #output["upper_simple"] = np.e**(output["coef_simple"] + output["conf_95_simple"]) - 1
    #output["upper_full"] = np.e**(output["coef_full"] + output["conf_95_full"]) - 1

    for x, y, z in zip(["upper_simple", "upper_full"], ["coef_simple", "coef_full"], ["conf_95_simple", "conf_95_full"]): output[x] = np.e**(output[y] + output[z]) - 1

    for x, y, z in zip(["lower_simple", "lower_full"], ["coef_simple", "coef_full"], ["conf_95_simple", "conf_95_full"]): output[x] = np.e**(output[y] - output[z]) - 1

    #output["lower_simple"] = np.e**(output["coef_simple"] - output["conf_95_simple"]) - 1
    #output["lower_full"] = np.e**(output["coef_full"] - output["conf_95_full"]) - 1

    output["coef_simple"] = np.e**output["coef_simple"] - 1
    output["coef_full"] = np.e**output["coef_full"] - 1

    for x, y  in zip(["upper_simple", "upper_full"], ["coef_simple", "coef_full"]): output[x] = output[x] - output[y]
    for x, y  in zip(["lower_simple", "lower_full"], ["coef_simple", "coef_full"]): output[x] = output[y] - output[x]

    output_index = list(output.index)

    my_range=np.array(list(range(1,len(output_index)+1)))

    output_index = [x[:16] for x in output.index]
    
    for i in range(0, len(output_index), 2):
       output_index[i] = "<br>" + output_index[i]

    fig = go.Figure(data=[
        go.Bar(name="Unadjusted", 
               x=my_range, 
               y=output["coef_simple"], 
               marker_color="#d7eaf7", 
               #opacity=0.2, 
               showlegend=False, 
               width=min(0.28, 0.28*len(my_range)/4), 
               hoverinfo="skip"),
        go.Bar(name="w/ Controls", 
               x=my_range, 
               y=output["coef_full"], 
               marker_color="#ffcdcd", 
               #opacity=0.2, 
               showlegend=False, 
               width=min(0.28, 0.28*len(my_range)/4), 
               hoverinfo="skip"),
        go.Scatter(name="Unadjusted", 
                   x=my_range-0.2, 
                   y=output["coef_simple"], 
                   mode="markers", 
                   marker_color="#3493d3", 
                   marker_size=20, 
                   error_y=dict(type="data", 
                                symmetric=False, 
                                array=output["upper_simple"], 
                                arrayminus=output["lower_simple"], 
                                width=5, 
                                thickness=2, 
                                visible=True),
                   hovertemplate="%{text}",
                   text=[name.replace("<br>", "") + 
                         ": {:,.1%}".format(coef) + 
                         ", 95% conf: [{:,.1%}".format(lower) + 
                         ", {:,.1%}]".format(upper) for name, coef, upper, lower in zip(output_index, 
                                                                                        output["coef_simple"], 
                                                                                        output["coef_simple"] + output["upper_simple"], 
                                                                                        output["coef_simple"] - output["lower_simple"])]),
        go.Scatter(name="w/ Controls", 
                   x=my_range+0.2, 
                   y=output["coef_full"], 
                   mode="markers", 
                   marker_color="red", 
                   marker_size=20,
                   error_y=dict(type="data", 
                                symmetric=False, 
                                array=output["upper_full"], 
                                arrayminus=output["lower_full"], 
                                width=5, 
                                thickness=2, 
                                visible=True),
                   hovertemplate="%{text}",
                   text=[name.replace("<br>", "") + 
                        ": {:,.1%}".format(coef) + 
                        ", 95% conf: [{:,.1%}".format(lower) + 
                        ", {:,.1%}]".format(upper) for name, coef, upper, lower in zip(output_index, 
                                                                                       output["coef_full"], 
                                                                                       output["coef_full"] + output["upper_full"], 
                                                                                       output["coef_full"] - output["lower_full"])])],
                   layout=go.Layout(title=go.layout.Title(text=title)))

    fig.update_layout(barmode="group", 
                      bargroupgap=0.1,
                      xaxis = dict(tickmode = "array",
                                   tickvals = my_range,
                                   ticktext = output_index,
                                   tickangle = 0,
                                   ticks = "outside",
                                   showline=True,
                                   linewidth=0.5,
                                   linecolor="black"),
                      yaxis = dict(tickformat = ".1%",
                                   ticks = "outside",
                                   title = "Pay Gap vs. " + controls[category]["omitted"][:16],
                                   showline=True,
                                   linewidth=0.5,
                                   linecolor="black",
                                   showgrid=True,
                                   gridcolor="#e8e8e8",
                                   gridwidth=0.5,
                                   zeroline=True,
                                   zerolinecolor="#e8e8e8",
                                   zerolinewidth=2),
                      width=max(750, len(my_range) * 100),
                      plot_bgcolor="white",
                      title_font = dict(size=20),
                      legend=go.layout.Legend(x=0,
                                              y=1.13,
                                              orientation="h"))

    if not os.path.exists("images"):
        os.mkdir("images")

    fig.write_image("images/" + category + ".jpg", scale=2)

    fig.show()
    
    return output, est_full

In [179]:
def prune_df(df: pd.DataFrame, category: str) -> pd.DataFrame:
    """
    Prunes dataframe columns to only include those with "category_" in the name
    
    Returns: Pruned dataframe
    """

    matching = [x for x in df.columns if category+"_" in x]

    return df[matching].copy()

In [180]:
def explain(df: pd.DataFrame, 
            regression, 
            controls: dict, 
            cat_to_explain: str, 
            coef_to_explain: str, 
            top: int = 5) -> (pd.DataFrame, float):
    """
    Decomposes explained portion of income gap between groups and returns top explainers

    Decomposition follows methodology of (Gelbach 2016)

    Returns: Dataframe with top 3 explainers and respective explained gaps
    """

    name = cat_to_explain + "_" + coef_to_explain

    df_copy = df.copy()

    X_1 = prune_df(df_copy, cat_to_explain).assign(const=1)

    explainers = [control for control in controls.keys() if control != cat_to_explain]

    results = pd.DataFrame()
    
    for i, explainer in enumerate(explainers):

        X_2 = prune_df(df_copy, explainer)

        X_2 = X_2.reindex(sorted(X_2.columns), axis=1)

        B_2 = regression.params.filter(like=explainer+"_").sort_index()

        X_2 = X_2[B_2.index]

        H_k = X_2.values @ B_2

        if i == 0:
            results = pd.DataFrame(sm.OLS(endog=H_k, exog=X_1).fit().params.rename(explainer))
        else:
            d = sm.OLS(endog=H_k, exog=X_1).fit().params.rename(explainer)

            results = results.join(d)

    neg = results.T.filter(like=name).sort_values(by=name)[name].sum() < 0

    results = results.T.filter(like=name).sort_values(by=name, ascending=neg)

    results.columns = [coef_to_explain]

    return results.iloc[:top].append(results.iloc[top:].sum().rename("Other")), regression.params.filter(like=name).values[0]

In [181]:
def output_waterfall(df: pd.DataFrame, 
                     coef_full: float, 
                     controls: dict, 
                     category: str,
                     exp: str) -> None:
    """
    Plots...


    """

    coef_simple = coef_full + df.values.flatten().sum()

    step_names = ["w/ Controls"] + list(df.index) + ["Unadjusted"]

    steps = np.array([coef_full] + list(df.values.flatten()) + [coef_simple])

    new_steps = [0] * len(steps)

    for step in range(len(steps)):
        if step == 0:
            new_steps[step] = np.e**steps[step] - 1
        elif step == len(steps) - 1:
            new_steps[step] = np.e**coef_simple - 1
        else:
            new_steps[step] = np.e**sum(steps[:step+1]) - 1 - sum(new_steps[:step])

    steps = new_steps

    my_range=np.array(list(range(1, len(steps)+1)))

    for i in range(0, len(step_names), 2):
       step_names[i] = "<br>" + step_names[i]

    fig = go.Figure(go.Waterfall(orientation="v",
                                 measure=["absolute"] + (["relative"] * len(df.index)) + ["total"],
                                 x=step_names,
                                 text=["{:,.1%}".format(steps[0])] + 
                                      ["{:+,.1%}".format(step) for step in steps[1:-1]] + 
                                      ["{:,.1%}".format(steps[-1])],
                                 textposition="outside",
                                 y=steps,
                                 connector = {"line":{"color":"grey"}},
                                 hovertemplate="%{text}",
                                 name=output.columns[0],
                                 totals = {"marker":{"color":"#3493d3"}},
                                 decreasing = {"marker":{"color":"#ff4747"}}))

    fig.update_layout(title = "Drivers of the " + output.columns[0][:16] + " pay gap", 
                      xaxis = dict(tickmode = "array",
                                   tickvals = step_names,
                                   tickangle = 0,
                                   ticks = "outside",
                                   showline=True,
                                   linewidth=0.5,
                                   linecolor="black"),
                      yaxis = dict(tickformat = ".1%",
                                   ticks = "outside",
                                   title = "Pay Gap vs. " + controls[category]["omitted"][:16],
                                   showline=True,
                                   linewidth=0.5,
                                   linecolor="black",
                                   showgrid=True,
                                   gridcolor="#e8e8e8",
                                   gridwidth=0.5,
                                   zeroline=True,
                                   zerolinecolor="#e8e8e8",
                                   zerolinewidth=2),
                      width=max(500, len(my_range) * 100),
                      plot_bgcolor="white",
                      title_font = dict(size=20))

    if not os.path.exists("images"):
        os.mkdir("images")

    fig.write_image("images/" + category + "_" + exp + "_waterfall.jpg", scale=2)

    fig.show()

In [216]:
test, est = output_graph(data_2019, controls, True, "SocialMedia", "race", "Income vs. Reddit")

coef_full  conf_95_full  coef_simple  \
SocialMedia_facebook                 -0.015267      0.019653     0.041365   
SocialMedia_youtube                  -0.013109      0.021865    -0.034621   
SocialMedia_twitter                   0.009364      0.018632     0.113439   
SocialMedia_instagram                 0.022789      0.024117     0.038481   
SocialMedia_no_answer                -0.013137      0.034999     0.026082   
SocialMedia_wechat_微信                 0.048778      0.133984     0.205810   
SocialMedia_whatsapp                 -0.020592      0.040291     0.133886   
SocialMedia_i_dont_use_social_media   0.000856      0.024483     0.046579   
SocialMedia_linkedin                  0.017099      0.031104     0.149078   
SocialMedia_snapchat                 -0.024278      0.063278    -0.159348   
SocialMedia_vk_вконта́кте            -0.300622      0.303148     0.019054   

                                     conf_95_simple  
SocialMedia_facebook                       0.025795  
Soci

In [218]:
cat = "SocialMedia"
exp = "whatsapp"

output, coef_full = explain(data_2019, est, controls, cat, exp, 5)

output_waterfall(output, coef_full, controls, cat, exp)

In [210]:
controls

{'Hobbyist': {'omitted': 'yes', 'controls': ['yes', 'no']},
 'Employment': {'omitted': 'full_time',
  'controls': ['full_time',
   'no_answer',
   'independent_contractor_freelancer_self_employed',
   'part_time']},
 'Student': {'omitted': 'no', 'controls': ['no']},
 'EdLevel': {'omitted': 'bachelors_degree_ba_bs_beng_',
  'controls': ['masters_degree_ma_ms_meng_mba_',
   'secondary_school_eg_american_high_school_german_realschule_gymnasium_',
   'professional_degree_jd_md_',
   'no_answer',
   'bachelors_degree_ba_bs_beng_',
   'primary_elementary_school',
   'other_doctoral_degree_phd_edd_',
   'associate_degree',
   'some_college_university_study_without_earning_a_degree']},
 'UndergradMajor': {'omitted': 'computer_science_computer_engineering_software_engineering',
  'controls': ['information_systems_information_technology_system_administration',
   'a_natural_science_ex_biology_chemistry_physics_',
   'a_social_science_ex_anthropology_psychology_political_science_',
   'another_en

In [214]:
output

,engineering_manager
YearsCodePro,0.068939
Age,0.019965
PurchaseWhat,0.011206
WorkWeekHrs,0.010666
ImpSyn,0.009674
Other,0.052460
